# CBEN 470 Microfluidics - Hydraulic Circuit Analysis

In [ ]:
import ipy_table
import numpy as np
from IPython.display import Image
from numpy import matrix
from numpy import linalg
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

Hydraulic circuit analysis (HCA) is used to design devices by estimating the fluidic network as an electrical circuit.

Recall that Poiseuille flow in a tube is:
$$u_z = \frac{1}{4\mu} \frac{\partial P}{\partial z}(R^2-r^2) $$

where $u_z$ is the fluid velocity in the axial direction, $\mu$ is the viscosity, $P$ is the pressure, $z$ is the axial direction, $R$ is the radius of the tube, and $r$ is the radial position.

This give the parabolic profile you've come to know for tube flow:

Integrate over the area to get the flow rate:
$$Q = -\frac{\pi R^4}{8 \mu} \frac{\partial P}{\partial z}$$
where $Q$ is the flow rate.

If flow is axial and the pressure gradient is uniform then:

$$ \frac{\partial P}{\partial z} \approx \frac{\Delta P}{L}$$

$$Q = \frac{\pi R^4}{8 \mu L}\Delta P$$

where $L$ is the length of the tube. Note that this approximation (partial derivative to delta) only holds for staight and long tubes. Here, long means that R/L << 1.


We rewrite the flow rate equation for a generic cross-sectional area:

$$Q = \frac{A R^4}{8 \mu L} \Delta P$$

We define the **hydraulic resitance**, $R_h$ as:

$$ R_h = \frac{8 \mu L}{A R^2}$$

$$ Q = \frac{\Delta P}{R_h} $$

This last equation is known as the **Hagen-Poiseuille Law**.

Note, that the Hagen-Poiseuille law is analogous to Ohm's Law:
$$I = \frac{\Delta V}{R}$$
where $I$ is the current, $V$ is the voltage, and $R$ is the resistance.

The image below show the equivalent 'hydraulic circuit'

<img src="HCA.png">


In [51]:
from ipy_table import *
circuits = [
    ['Electrical Circuit', 'Fluidic Circuit'],
    ['Battery/Power Supply', 'Pump'],
    ['$\Delta V$', '$\Delta P$'],
    ['I', 'Q'],
    ['i','u'],
    ['R','R$_h$'],
    ['C','C$_h$'],
    ['L, inductance','inertia (usually negligible)']

];
make_table(circuits)
set_row_style(0, bold=True)

Electrical Circuit,Fluidic Circuit
Battery/Power Supply,Pump
$\Delta V$,$\Delta P$
I,Q
i,u
R,R$_h$
C,C$_h$
"L, inductance",inertia (usually negligible)


Consider several tubes that intersect at a node. Conservation of mass requires that:

$$\sum_{channels} Q = 0$$

Using the Hagen-Poiseuille law and conservation of mass gives steady-state behavior.

We have assumed circular cross-section, but microchannels are usually rectangular, trapezoidal, or semicircular.

For a circular channel the hydraulic radius, R_h

$$R_h = \frac{8 \eta L}{R_h^2A}$$ 

In general: 

$$R_h = \frac{2A}{P}$$

where $A$ is the area and $P$ is the perimeter. 


**Series and parallel components rules**

Resistors in Series: $R_h = R_{h,1} + R_{h,2}$

Resistors in Parallel: $\frac{1}{R_h} = \frac{1}{R_{h,1}} + \frac{1}{R_{h,2}}$

**Example: Bifurcating channel**

<img src="BifurcatingChan.png">

All channels have a radius, R = 20 $\mu$m

L1 = 0.5 cm

L2 = 1.0 cm

L3 = 2.0 cm

P1 = 0.11 MPa

P3 = 0.1 MPa

P4 = 0.1 MPa

$\mu$=10$^{-3}$ Pa s

Unknowns: 

N - nodes (including inlets and outlets) -> Use conservation of mass

M - channels --> Hagen-Poiseuille

We need a (N + M) X (N + M) matrix of equations which can be inverted to obtain the solution. 

For this examples, we have seven unknowns--P1, P2, P3, P4, Q1, Q2, Q3--thus, we need seven equations.

First, find the hydraulic resistance of the three channels:

$$R_h = \frac{8 \mu L}{\pi R^4}$$

Calcualte the resistances in SI units:

In [52]:
L1=0.5e-2; L2=1e-2; L3=2e-2; mu=1e-3; R=20e-6;
R1=8*mu*L1/(np.pi*R**4) #Units N/s m^5
R2=8*mu*L2/(np.pi*R**4)
R3=8*mu*L3/(np.pi*R**4)

Convert Resistances from SI units to Pa/s um^3, and Pressures to Pa, so that numerical inversion works better

In [53]:
R11 = R1*1e-18; R22 = R2*1e-18; R33 = R3*1e-18; P1=110000; P2=100000; P3=100000

Create the coefficient matrix

In [54]:
A = matrix( [[1, 0, 0, 0, 0, 0, 0], \
             [0, 0, 1, 0, 0, 0, 0], \
             [0, 0, 0, 1, 0, 0, 0], \
             [1, -1, 0, 0, -R11, 0, 0], \
             [0, 1, -1, 0, 0, -R22, 0], \
             [0, 1, 0, -1, 0, 0, -R33], \
            [0, 0, 0, 0, 1, -1, -1]])

Create the solution matrix

In [55]:
b = matrix([[P1],[P2],[P3],[0],[0],[0],[0]])

Solve the unknown by matrix inversion

In [56]:
print linalg.solve(A,b)

[[   110000.        ]
 [   105714.28571429]
 [   100000.        ]
 [   100000.        ]
 [ 53855874.06153931]
 [ 35903916.0410262 ]
 [ 17951958.02051311]]


From the inversion we find that P2 = 105,714 Pa, and the flow rates are Q1 = 0.054 uL/sec (5.4 x 10$^7$ um$^3$/s), Q2 = 0.036 uL/sec, Q3 = 0.018 uL/sec